# Millenials and Investment: an Ongoing Exploration

In case you haven't heard, [millenials are killing everything](http://www.businessinsider.com/millennials-are-killing-list-2017-8/#diamonds-12) from diamonds to department stores to designer crap to grocery chains.

Why? Sure, the recession had an impact. But also, millenials pay more attention to ethics than many multinational corporations bargain for. They cite the blood diamond trade as a major reason to spring for non-traditional engagement rings. They opt for grocery providers that can tell them where their food is coming from and under what conditions it was produced. They're ditching the fast fashion industry for higher-priced items purchased secondhand on sites like Poshmark and ThreadUp. 

And as millenials reach the age where they might accrue some savings, it makes sense that they would care about where that is going, too. In addition to millenial attendance at the NoDAPL protests, we saw thousands of millenials divest from Western Union, Bank of America, and other banks that loaned money to the project. Maybe megacorps won't change their tunes because a few thousand people stood in a field to get mowed down by water cannons, but they're more likely to sit up and listen when those same people take their hard-earned doll hairs to another playhouse.

So we see that millenials are surveying their options to spend and save according to their values. What about investing? Any personal finance 101 that isn't taught by a financial advisor will recommend a low cost index as the place to stick extra money so it can grow with the market. Most index funds, including the most recommended one (Vanguard), decide their investments via index-matching: matching their holdings to the S&P500 by market cap, with no other variables. Thing is, plenty of investors are expressing interest in taking ethical considerations into account. Some portfolios do this by blanket blocking investments in certain industries like tobacco or porn. Other more advanced optsions, like Betterment's AutoSRI portfolio, use actual ESG data to determine where they invest the money. There isn't (yet) a fully customizable option to allow folks to automatically invest their funds based on a checklist of their individual values. For a while, I've thought about building a toy version of what that might look like. 

When I talk about the idea with friends and relatives, I get the following objection: 'What about the returns?' Touche. Nobody wants to lose out on their potential earnings. At first, I figured I'd build a tolerance into the system that allowed investors to say 'These are my values, but please don't invest in a way that will trail general market performance by more than x percent.' The algorithm would then predict stock performance for each company, somehow blend that with ESG rating, and come up with a combined weight for divvying up investment money.

Before I build that, though, I need to test the assumption that high ESG ratings *do* correlate negatively with returns. If they don't, there's no need for the tolerance measure in the first place.

I'm not the first person to run correlations along these lines. Dorfleitner, Utz, and Wimmer published a paper on this just last year. [Their analysis](http://www.tandfonline.com/doi/full/10.1080/20430795.2017.1403272?scroll=top&needAccess=true) suggests that higher corporate social responsibility ratings *increase* returns over a long period of time ("long" being a 12 year period from 2002-2014). They even identify three specific areas that correlate with higher than average returns: emission and resource reduction, workforce, and society. So in my exploration, I'll dig into some specific CSR breakdowns with the data I have on S&P 500 companies. 

In [50]:
import pandas as pd

## Correlating KLD ESG Ratings to Stock Performance, 1991-1999

Let's determine whether we notice any correlation between companies' environmental, social, and governmental ratings and their stock performance during the 1990s. 

### First, we pull in the ESG data. 

These come from KLD and are now distributed by MSGI. I pulled them from an academic database. Don't rerun this notebook because I didn't push the actual data to Github. 

In [4]:
data = pd.read_excel('../stockproject/12231046.1990-1999.stats/1991 HistoricalSpreadsheet_STATS.xls')

In [8]:
sp500 = data[data['SP500'] == True]
sp500.head()

,Ticker,CompanyName,CUSIP,SP500,DS400,Russell1000,LCS,Russell2000,BMS,COM-str-A,...,HUM-str-#,HUM-con-#,PRO-str-#,PRO-con-#,ALC-con-#,GAM-con-#,TOB-con-#,FIR-con-#,MIL-con-#,NUC-con-#
0,ABT,Abbott Laboratories,NaN,True,False,NaN,NaN,NaN,NaN,0,...,NR,1,0,1,0,0,0,NR,0,0
1,AMT,Acme-Cleveland Corporation,NaN,True,False,NaN,NaN,NaN,NaN,0,...,NR,0,0,0,0,0,0,NR,0,1
3,AMD,"Advanced Micro Devices, Inc.",NaN,True,True,NaN,NaN,NaN,NaN,0,...,NR,0,0,0,0,0,0,NR,1,0
4,AET,Aetna Life & Casualty Company,NaN,True,True,NaN,NaN,NaN,NaN,0,...,NR,0,0,1,0,0,0,NR,0,0
6,AHM,Ahmanson & Company,NaN,True,True,NaN,NaN,NaN,NaN,0,...,NR,0,0,0,0,0,0,NR,0,0


### Second, we pull in stock performance data.

This data contains stock returns by quarter for S&P500 companies dating back to 1979. We'll pull the columns for the '90s for now.

In [51]:
data = pd.read_excel('../stockproject/Cleaned_Researcher_Dataset.xlsx')

In [55]:
new_header = data.iloc[0] #grab the first row for the header
content = data[1:] #take the data less the header row
content.columns = new_header #set the header row as the df header
content.head()

tickers = content.iloc[:,0:2]
tickers.columns = list(new_header)[0:2]

dates = content.iloc[:,45:92]
dates.columns = list(new_header)[45:92]

result = pd.concat([tickers, dates], axis=1)
# print(new_header)
result.head()

,Ticker,Company Name,1989-12-31 00:00:00,1990-03-31 00:00:00,1990-06-30 00:00:00,1990-09-30 00:00:00,1990-12-31 00:00:00,1991-03-31 00:00:00,1991-06-30 00:00:00,1991-09-30 00:00:00,...,1999-03-31 00:00:00,1999-06-30 00:00:00,1999-09-30 00:00:00,1999-12-31 00:00:00,2000-03-31 00:00:00,2000-06-30 00:00:00,2000-09-30 00:00:00,2000-12-31 00:00:00,2001-03-31 00:00:00,2001-06-30 00:00:00
1,A,Agilent Technologies Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,73.75,48.94,54.75,30.73,32.5
2,AA,Alcoa Inc,75,64.75,63.75,62.63,57.63,65.5,67.5,63.88,...,41.19,61.88,62.06,83,70.25,29,25.31,33.5,35.95,39.4
3,AAL,American Airlines Group,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAL (Alex),Alexander & Alexander,30.63,26.75,24.38,17.88,23.13,26.25,22.5,20.88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AAP,Advance Auto Parts Inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Third, we combine the data into one dataframe.

We find all the companies for which we have both stock price data and ESG data, and we put the information together.

In [65]:
tickers = list(sp500["Ticker"]) #get all the company tickers for which we have esg data
prices_for_esg_companies = result[result["Ticker"].isin(tickers)] #get the stock data from companies in that list
prices_for_esg_companies.shape

(399, 49)

Hm, we have 399 of them. 100 are missing. Why?

In [67]:
relevant_esgs = sp500[sp500["Ticker"].isin(list(prices_for_esg_companies["Ticker"]))]
relevant_esgs.shape

(399, 121)

In [69]:
all_data = pd.concat([relevant_esgs, prices_for_esg_companies], axis = 1) #put the esg and stock data in one dataframe
all_data.head()

,Ticker,CompanyName,CUSIP,SP500,DS400,Russell1000,LCS,Russell2000,BMS,COM-str-A,...,1999-03-31 00:00:00,1999-06-30 00:00:00,1999-09-30 00:00:00,1999-12-31 00:00:00,2000-03-31 00:00:00,2000-06-30 00:00:00,2000-09-30 00:00:00,2000-12-31 00:00:00,2001-03-31 00:00:00,2001-06-30 00:00:00
0,ABT,Abbott Laboratories,NaN,True,False,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMT,Acme-Cleveland Corporation,NaN,True,False,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.19,61.88,62.06,83,70.25,29,25.31,33.5,35.95,39.4
3,AMD,"Advanced Micro Devices, Inc.",NaN,True,True,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AET,Aetna Life & Casualty Company,NaN,True,True,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
#let's do some cleanup so we only have the columns we need.
clean_data = all_data.drop(['CUSIP', 'SP500', 'DS400', 'Russell1000', 'LCS', 'Russell2000', 'BMS'], axis = 1)
clean_data.head()

,Ticker,CompanyName,COM-str-A,COM-str-B,COM-str-C,COM-str-D,COM-str-F,COM-str-G,COM-str-X,COM-con-A,...,1999-03-31 00:00:00,1999-06-30 00:00:00,1999-09-30 00:00:00,1999-12-31 00:00:00,2000-03-31 00:00:00,2000-06-30 00:00:00,2000-09-30 00:00:00,2000-12-31 00:00:00,2001-03-31 00:00:00,2001-06-30 00:00:00
0,ABT,Abbott Laboratories,0.0,0.0,0.0,NR,NR,NR,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AMT,Acme-Cleveland Corporation,0.0,0.0,0.0,NR,NR,NR,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.19,61.88,62.06,83,70.25,29,25.31,33.5,35.95,39.4
3,AMD,"Advanced Micro Devices, Inc.",0.0,0.0,0.0,NR,NR,NR,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AET,Aetna Life & Casualty Company,0.0,1.0,1.0,NR,NR,NR,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#Now let's strip this down so we can calculate returns over the course of the ten year period.
clean_data.columns.values

array(['Ticker', 'CompanyName', 'COM-str-A', 'COM-str-B', 'COM-str-C',
       'COM-str-D', 'COM-str-F', 'COM-str-G', 'COM-str-X', 'COM-con-A',
       'COM-con-B', 'COM-con-D', 'COM-con-X', 'CGOV-str-A', 'CGOV-str-C',
       'CGOV-str-D', 'CGOV-str-E', 'CGOV-str-X', 'CGOV-con-B',
       'CGOV-con-F', 'CGOV-con-G', 'CGOV-con-H', 'CGOV-con-I',
       'CGOV-con-X', 'DIV-str-A', 'DIV-str-B', 'DIV-str-C', 'DIV-str-D',
       'DIV-str-E', 'DIV-str-F', 'DIV-str-G', 'DIV-str-X', 'DIV-con-A',
       'DIV-con-B', 'DIV-con-X', 'EMP-str-A', 'EMP-str-B', 'EMP-str-C',
       'EMP-str-D', 'EMP-str-F', 'EMP-str-G', 'EMP-str-X', 'EMP-con-A',
       'EMP-con-B', 'EMP-con-C', 'EMP-con-D', 'EMP-con-X', 'ENV-str-A',
       'ENV-str-B', 'ENV-str-C', 'ENV-str-D', 'ENV-str-F', 'ENV-str-X',
       'ENV-con-A', 'ENV-con-B', 'ENV-con-C', 'ENV-con-D', 'ENV-con-E',
       'ENV-con-F', 'ENV-con-X', 'HUM-str-A', 'HUM-str-D', 'HUM-str-G',
       'HUM-str-X', 'HUM-con-A', 'HUM-con-B', 'HUM-con-C', 'HUM-con-D',
       '

In [77]:
esg_columns = clean_data.iloc[:,0:114]
esg_columns.columns.values

array(['Ticker', 'CompanyName', 'COM-str-A', 'COM-str-B', 'COM-str-C',
       'COM-str-D', 'COM-str-F', 'COM-str-G', 'COM-str-X', 'COM-con-A',
       'COM-con-B', 'COM-con-D', 'COM-con-X', 'CGOV-str-A', 'CGOV-str-C',
       'CGOV-str-D', 'CGOV-str-E', 'CGOV-str-X', 'CGOV-con-B',
       'CGOV-con-F', 'CGOV-con-G', 'CGOV-con-H', 'CGOV-con-I',
       'CGOV-con-X', 'DIV-str-A', 'DIV-str-B', 'DIV-str-C', 'DIV-str-D',
       'DIV-str-E', 'DIV-str-F', 'DIV-str-G', 'DIV-str-X', 'DIV-con-A',
       'DIV-con-B', 'DIV-con-X', 'EMP-str-A', 'EMP-str-B', 'EMP-str-C',
       'EMP-str-D', 'EMP-str-F', 'EMP-str-G', 'EMP-str-X', 'EMP-con-A',
       'EMP-con-B', 'EMP-con-C', 'EMP-con-D', 'EMP-con-X', 'ENV-str-A',
       'ENV-str-B', 'ENV-str-C', 'ENV-str-D', 'ENV-str-F', 'ENV-str-X',
       'ENV-con-A', 'ENV-con-B', 'ENV-con-C', 'ENV-con-D', 'ENV-con-E',
       'ENV-con-F', 'ENV-con-X', 'HUM-str-A', 'HUM-str-D', 'HUM-str-G',
       'HUM-str-X', 'HUM-con-A', 'HUM-con-B', 'HUM-con-C', 'HUM-con-D',
       '

In [82]:
start_stock_columns = clean_data.iloc[:,116:117]
start_stock_columns.columns.values

array([Timestamp('1989-12-31 00:00:00')], dtype=object)

In [89]:
end_stock_columns = clean_data.iloc[:,156:157]
end_stock_columns.columns.values

array([Timestamp('1999-12-31 00:00:00')], dtype=object)

In [92]:
minimum_data = pd.concat([esg_columns, start_stock_columns, end_stock_columns], axis=1)
minimum_data.shape
minimum_data.head()

,Ticker,CompanyName,COM-str-A,COM-str-B,COM-str-C,COM-str-D,COM-str-F,COM-str-G,COM-str-X,COM-con-A,...,PRO-str-#,PRO-con-#,ALC-con-#,GAM-con-#,TOB-con-#,FIR-con-#,MIL-con-#,NUC-con-#,1989-12-31 00:00:00,1999-12-31 00:00:00
0,ABT,Abbott Laboratories,0.0,0.0,0.0,NR,NR,NR,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,NR,0.0,0.0,NaN,NaN
1,AMT,Acme-Cleveland Corporation,0.0,0.0,0.0,NR,NR,NR,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NR,0.0,1.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,83
3,AMD,"Advanced Micro Devices, Inc.",0.0,0.0,0.0,NR,NR,NR,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NR,1.0,0.0,NaN,NaN
4,AET,Aetna Life & Casualty Company,0.0,1.0,1.0,NR,NR,NR,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,NR,0.0,0.0,NaN,NaN
